In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/home/ubuntu/repos/learning-habits-analysis')
from utils.data import Subject, load_participant_list

In [3]:
base_dir = '/home/ubuntu/data/learning-habits'
bids_dir = "/home/ubuntu/data/learning-habits/bids_dataset/derivatives/fmriprep-24.0.1"

sub_ids = load_participant_list(base_dir)

In [5]:
#subjects = [Subject(base_dir, sub_id, include_modeling=True, include_imaging=True, bids_dir=bids_dir) for sub_id in sub_ids]
subject = Subject(base_dir, '01', include_modeling=True, include_imaging=True, bids_dir=bids_dir)

In [10]:
subject.load_confounds(run='test', include_cos=False)[0]

,csf,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,white_matter
0,7.528526,-0.003160,0.001261,-0.000766,0.035159,0.206432,-0.363745,4.508462
1,19.601565,-0.001440,0.001088,-0.000840,0.040926,0.175414,-0.225295,4.327591
2,-7.243666,-0.000051,0.000935,-0.000766,0.038637,0.130787,-0.208452,0.263054
3,16.888935,-0.000637,0.001163,-0.000766,0.057044,0.172456,-0.190308,3.497631
4,6.986960,0.000111,0.001087,-0.000766,0.038848,0.190303,-0.160092,1.398740
...,...,...,...,...,...,...,...,...
588,1.819011,-0.000886,0.000753,0.000655,0.027877,-0.272713,0.044876,3.298673
589,0.055517,-0.003235,-0.000334,0.000423,0.004661,-0.189705,0.005532,5.788290
590,13.113162,-0.001705,0.000035,0.000423,-0.014014,-0.076701,0.084327,2.082381
591,15.301813,-0.004755,-0.000304,0.000661,-0.032191,0.012379,-0.047329,-2.375171


In [11]:
run = 'test'
columns_event = {'first_stim_value_rl':'first_stim_presentation',
                'first_stim_value_ck':'first_stim_presentation',
                'first_stim':'first_stim_presentation'}
# Load events
events = getattr(subject, run).extend_events_df(columns_event)


# Handle the exclusion of highest/lowest value stimuli

events['trial_type'] = events.apply(
    lambda row: f"{row['trial_type']}_{'exclude' if int(row['first_stim']) in (1, 8) else 'include'}"
    if row['trial_type'] == 'first_stim_presentation' else row['trial_type'],
    axis=1
) 

In [12]:
events

,onset,duration,trial_type,trial,first_stim_value_rl,first_stim_value_ck,first_stim
0,0.002346,0.813894,first_stim_presentation_exclude,1,1.000017,0.042455,1
1,0.816239,0.550671,second_stim_presentation,1,0.000000,0.000000,None
2,1.366910,0.000000,response,1,0.000000,0.000000,None
3,1.370405,1.951056,purple_frame,1,0.000000,0.000000,None
4,3.321461,6.203388,iti,1,0.000000,0.000000,None
...,...,...,...,...,...,...,...
674,1360.803064,0.859366,first_stim_presentation_include,136,2.999963,0.137329,5
675,1361.662430,0.730383,second_stim_presentation,136,0.000000,0.000000,None
676,1362.392813,0.000000,response,136,0.000000,0.000000,None
677,1362.394750,1.770824,purple_frame,136,0.000000,0.000000,None


In [6]:
bbt = pd.concat(
    [pd.concat([pd.DataFrame({'sub_id': [sub.sub_id] * len(sub.extended_trials)}), sub.extended_trials.reset_index(drop=True)], axis=1) for sub in subjects],
    ignore_index=True
)

In [24]:
bbt.to_csv('/home/ubuntu/data/learning-habits//bbt.csv', index=False)

In [5]:
run = 'test'

In [6]:
subject.learning1.extend_events_df({'first_stim_cat': 'first_stim_presentation'})

,onset,duration,trial_type,trial,first_stim_cat
0,0.009399,0.817157,first_stim_presentation,1,house
1,0.826556,0.934432,second_stim_presentation,1,None
2,1.760988,0.000000,response,1,None
3,1.766016,1.569584,purple_frame,1,None
4,2.268539,1.067061,points_feedback,1,None
...,...,...,...,...,...
565,959.906510,0.518548,second_stim_presentation,96,None
566,960.425058,0.000000,response,96,None
567,960.426145,1.983533,purple_frame,96,None
568,960.927272,1.482406,points_feedback,96,None


In [8]:
subject.trials.columns

Index(['block', 'left_stim', 'right_stim', 'left_value', 'right_value',
       'shift', 'action', 'rt', 'chosen_stim', 'reward', 'correct',
       't_first_stim', 't_second_stim', 't_action', 't_purple_frame',
       't_points_feedback', 't_iti_onset', 't_trial_end', 'first_stim',
       'second_stim', 'left_stim_name', 'right_stim_name', 'left_stim_cat',
       'right_stim_cat', 'first_stim_name', 'second_stim_name',
       'first_stim_cat', 'second_stim_cat'],
      dtype='object')

In [51]:
# Load events
events = getattr(subject, run).extend_events_df(columns_event={'first_stim_cat': 'first_stim_presentation'})

# Add stim category to first stim presentation
events['trial_type'] = events.apply(
    lambda row: f"{row['trial_type']}_{row['first_stim_cat']}" if row['trial_type'] == 'first_stim_presentation' else row['trial_type'],
    axis=1
)

In [52]:
events

,onset,duration,trial_type,trial,first_stim_cat
0,0.002346,0.813894,first_stim_presentation_figure,1,figure
1,0.816239,0.550671,second_stim_presentation,1,None
2,1.366910,0.000000,response,1,None
3,1.370405,1.951056,purple_frame,1,None
4,3.321461,6.203388,iti,1,None
...,...,...,...,...,...
674,1360.803064,0.859366,first_stim_presentation_hand,136,hand
675,1361.662430,0.730383,second_stim_presentation,136,None
676,1362.392813,0.000000,response,136,None
677,1362.394750,1.770824,purple_frame,136,None
